# AUTOARIMA

In [ ]:
import pandas as pd
import numpy as np

ruta_archivo = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo\SI_SO Ciclo Sep-2024AE.csv"
df = pd.read_csv(ruta_archivo, delimiter=',', decimal=',', encoding='utf-8').rename(columns={'Canal': 'Canal 3', 'Cantidad': 'Venta', 'idSKU': 'Ultimo Eslabón'})
df['ID'] = df['Ultimo Eslabón'].astype(str) + df['Canal 3'].astype(str)
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d')
df = df.sort_values(by=['Ultimo Eslabón', 'Fecha']).reset_index(drop=True)
pivot_table = df.pivot_table(index=['Ultimo Eslabón', 'Canal 3'], columns='Fecha', values='Venta', fill_value=0).reset_index()
pivot_table = pivot_table[pivot_table.iloc[:, 2:].sum(axis=1) != 0]
ultima_fecha = pivot_table.columns[-1]
primer_fecha = ultima_fecha - pd.DateOffset(months=23)
ventas_ultimos_24_meses = pivot_table.loc[:, primer_fecha:ultima_fecha]
intervalos = ventas_ultimos_24_meses.apply(lambda x: (x != 0).astype(int).diff().fillna(1).abs().sum(), axis=1)
ventas_activas = (ventas_ultimos_24_meses != 0).sum(axis=1)
pivot_table['ADI'] = intervalos / ventas_activas
ventas_no_cero = ventas_ultimos_24_meses.replace(0, np.nan)
pivot_table['CV²'] = (ventas_no_cero.std(axis=1) / ventas_no_cero.mean(axis=1)) ** 2
def clasificar_demanda(row):
    if row['ADI'] < 1.32 and row['CV²'] < 0.49: return 'Smooth'
    elif row['ADI'] >= 1.32 and row['CV²'] < 0.49: return 'Intermittent'
    elif row['ADI'] < 1.32 and row['CV²'] >= 0.49: return 'Erratic'
    else: return 'Lumpy'
pivot_table['Demand Type'] = pivot_table.apply(clasificar_demanda, axis=1)
melted_data = pivot_table.melt(id_vars=['Ultimo Eslabón', 'Canal 3', 'ADI', 'CV²', 'Demand Type'], var_name='Fecha', value_name='Venta')
melted_data['Fecha'] = pd.to_datetime(melted_data['Fecha'], format='%Y-%m-%d')
data = melted_data
data.shape

## Intermittent

In [ ]:
data = melted_data[melted_data['Demand Type'] == 'Intermittent']
data = data[['Ultimo Eslabón', 'Canal 3', 'Fecha', 'Venta']]
data


In [ ]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import os

# Asumiendo que las fechas están en formato 'YYYY-MM-DD'
data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')  # Mantén el formato original

predictions = []

# Simulamos estar en diciembre de 2023 pero queremos predecir a partir de enero de 2024
# Usaremos todos los datos hasta julio de 2024
for id_value in data['ID'].unique():
    data_id = data[data['ID'] == id_value]
    data_id.set_index('Fecha', inplace=True, drop=True)
    data_id.index.freq = 'MS'  # Esto supone que las fechas son mensuales y consecutivas
    y = data_id['Venta']
    
    if y.isnull().any() or np.isinf(y).any():
        print(f'Serie temporal con valores nulos o infinitos para ID: {id_value}')
        continue
    
    try:
        # Ajustar AutoARIMA para encontrar el mejor modelo ARIMA
        model = auto_arima(y, seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)
        future_predictions = model.predict(n_periods=12)
        
        # Crear las fechas de predicción a partir de enero de 2024
        future_dates = pd.date_range(start='2024-01-01', periods=12, freq='MS').strftime('%Y-%m-%d')
        
        for date, pred in zip(future_dates, future_predictions):
            predictions.append({
                'ID': id_value,
                'Fecha': date,
                'Prediccion_Venta': pred
            })
    except Exception as e:
        print(f'Error al ajustar el modelo para ID: {id_value} - {e}')
        continue

predictions_df = pd.DataFrame(predictions)
output_dir = r'C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo'
output_file = os.path.join(output_dir, 'AutoArima_Intermittent.csv')
predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')

print(f'Predicciones guardadas en {output_file}')


## Erratic

In [ ]:
import pandas as pd
import numpy as np

ruta_archivo = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo\SI_SO Ciclo Sep-2024AE.csv"
df = pd.read_csv(ruta_archivo, delimiter=',', decimal=',', encoding='utf-8').rename(columns={'Canal': 'Canal 3', 'Cantidad': 'Venta', 'idSKU': 'Ultimo Eslabón'})
df['ID'] = df['Ultimo Eslabón'].astype(str) + df['Canal 3'].astype(str)
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d')
df = df.sort_values(by=['Ultimo Eslabón', 'Fecha']).reset_index(drop=True)
pivot_table = df.pivot_table(index=['Ultimo Eslabón', 'Canal 3'], columns='Fecha', values='Venta', fill_value=0).reset_index()
pivot_table = pivot_table[pivot_table.iloc[:, 2:].sum(axis=1) != 0]
ultima_fecha = pivot_table.columns[-1]
primer_fecha = ultima_fecha - pd.DateOffset(months=23)
ventas_ultimos_24_meses = pivot_table.loc[:, primer_fecha:ultima_fecha]
intervalos = ventas_ultimos_24_meses.apply(lambda x: (x != 0).astype(int).diff().fillna(1).abs().sum(), axis=1)
ventas_activas = (ventas_ultimos_24_meses != 0).sum(axis=1)
pivot_table['ADI'] = intervalos / ventas_activas
ventas_no_cero = ventas_ultimos_24_meses.replace(0, np.nan)
pivot_table['CV²'] = (ventas_no_cero.std(axis=1) / ventas_no_cero.mean(axis=1)) ** 2
def clasificar_demanda(row):
    if row['ADI'] < 1.32 and row['CV²'] < 0.49: return 'Smooth'
    elif row['ADI'] >= 1.32 and row['CV²'] < 0.49: return 'Intermittent'
    elif row['ADI'] < 1.32 and row['CV²'] >= 0.49: return 'Erratic'
    else: return 'Lumpy'
pivot_table['Demand Type'] = pivot_table.apply(clasificar_demanda, axis=1)
melted_data = pivot_table.melt(id_vars=['Ultimo Eslabón', 'Canal 3', 'ADI', 'CV²', 'Demand Type'], var_name='Fecha', value_name='Venta')
melted_data['Fecha'] = pd.to_datetime(melted_data['Fecha'], format='%Y-%m-%d')
data = melted_data
data.shape

In [ ]:
data = melted_data[melted_data['Demand Type'] == 'Erratic']
data = data[['Ultimo Eslabón', 'Canal 3', 'Fecha', 'Venta']]
data

In [ ]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import os

# Asumiendo que las fechas están en formato 'YYYY-MM-DD'
data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')  # Mantén el formato original

predictions = []

# Simulamos estar en diciembre de 2023 pero queremos predecir a partir de enero de 2024
# Usaremos todos los datos hasta julio de 2024
for id_value in data['ID'].unique():
    data_id = data[data['ID'] == id_value]
    data_id.set_index('Fecha', inplace=True, drop=True)
    data_id.index.freq = 'MS'  # Esto supone que las fechas son mensuales y consecutivas
    y = data_id['Venta']
    
    if y.isnull().any() or np.isinf(y).any():
        print(f'Serie temporal con valores nulos o infinitos para ID: {id_value}')
        continue
    
    try:
        # Ajustar AutoARIMA para encontrar el mejor modelo ARIMA
        model = auto_arima(y, seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)
        future_predictions = model.predict(n_periods=12)
        
        # Crear las fechas de predicción a partir de enero de 2024
        future_dates = pd.date_range(start='2024-01-01', periods=12, freq='MS').strftime('%Y-%m-%d')
        
        for date, pred in zip(future_dates, future_predictions):
            predictions.append({
                'ID': id_value,
                'Fecha': date,
                'Prediccion_Venta': pred
            })
    except Exception as e:
        print(f'Error al ajustar el modelo para ID: {id_value} - {e}')
        continue

predictions_df = pd.DataFrame(predictions)
output_dir = r'C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo'
output_file = os.path.join(output_dir, 'AutoArima_Erratic.csv')
predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')

print(f'Predicciones guardadas en {output_file}')


## Smooth

In [1]:
import pandas as pd
import numpy as np

ruta_archivo = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo\SI_SO Ciclo Sep-2024AE.csv"
df = pd.read_csv(ruta_archivo, delimiter=',', decimal=',', encoding='utf-8').rename(columns={'Canal': 'Canal 3', 'Cantidad': 'Venta', 'idSKU': 'Ultimo Eslabón'})
df['ID'] = df['Ultimo Eslabón'].astype(str) + df['Canal 3'].astype(str)
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d')
df = df.sort_values(by=['Ultimo Eslabón', 'Fecha']).reset_index(drop=True)
pivot_table = df.pivot_table(index=['Ultimo Eslabón', 'Canal 3'], columns='Fecha', values='Venta', fill_value=0).reset_index()
pivot_table = pivot_table[pivot_table.iloc[:, 2:].sum(axis=1) != 0]
ultima_fecha = pivot_table.columns[-1]
primer_fecha = ultima_fecha - pd.DateOffset(months=23)
ventas_ultimos_24_meses = pivot_table.loc[:, primer_fecha:ultima_fecha]
intervalos = ventas_ultimos_24_meses.apply(lambda x: (x != 0).astype(int).diff().fillna(1).abs().sum(), axis=1)
ventas_activas = (ventas_ultimos_24_meses != 0).sum(axis=1)
pivot_table['ADI'] = intervalos / ventas_activas
ventas_no_cero = ventas_ultimos_24_meses.replace(0, np.nan)
pivot_table['CV²'] = (ventas_no_cero.std(axis=1) / ventas_no_cero.mean(axis=1)) ** 2
def clasificar_demanda(row):
    if row['ADI'] < 1.32 and row['CV²'] < 0.49: return 'Smooth'
    elif row['ADI'] >= 1.32 and row['CV²'] < 0.49: return 'Intermittent'
    elif row['ADI'] < 1.32 and row['CV²'] >= 0.49: return 'Erratic'
    else: return 'Lumpy'
pivot_table['Demand Type'] = pivot_table.apply(clasificar_demanda, axis=1)
melted_data = pivot_table.melt(id_vars=['Ultimo Eslabón', 'Canal 3', 'ADI', 'CV²', 'Demand Type'], var_name='Fecha', value_name='Venta')
melted_data['Fecha'] = pd.to_datetime(melted_data['Fecha'], format='%Y-%m-%d')
data = melted_data
data.shape

(4701211, 7)

In [2]:
data = melted_data[melted_data['Demand Type'] == 'Smooth']
data = data[['Ultimo Eslabón', 'Canal 3', 'Fecha', 'Venta']]
data

,Ultimo Eslabón,Canal 3,ADI,CV²,Demand Type,Fecha,Venta
9,100481,CL RETAIL,0.041667,0.075945,Smooth,2018-01-01,63.0
12,100485,CL RETAIL,0.086957,0.258048,Smooth,2018-01-01,0.0
13,100486,CL RETAIL,0.111111,0.091449,Smooth,2018-01-01,0.0
14,100487,CL RETAIL,0.041667,0.266401,Smooth,2018-01-01,0.0
17,100491,CL RETAIL,0.111111,0.155451,Smooth,2018-01-01,0.0
...,...,...,...,...,...,...,...
4701197,1171461,CL RETAIL,1.000000,0.000000,Smooth,2024-07-01,3.0
4701200,1171471,CL RETAIL,1.000000,0.000000,Smooth,2024-07-01,1.0
4701202,1171474,CL RETAIL,1.000000,0.000000,Smooth,2024-07-01,1.0
4701204,1171481,CL RETAIL,1.000000,0.000000,Smooth,2024-07-01,2.0


In [3]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import os

# Asumiendo que las fechas están en formato 'YYYY-MM-DD'
data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')  # Mantén el formato original

predictions = []

# Simulamos estar en diciembre de 2023 pero queremos predecir a partir de enero de 2024
# Usaremos todos los datos hasta julio de 2024
for id_value in data['ID'].unique():
    data_id = data[data['ID'] == id_value]
    data_id.set_index('Fecha', inplace=True, drop=True)
    data_id.index.freq = 'MS'  # Esto supone que las fechas son mensuales y consecutivas
    y = data_id['Venta']
    
    if y.isnull().any() or np.isinf(y).any():
        print(f'Serie temporal con valores nulos o infinitos para ID: {id_value}')
        continue
    
    try:
        # Ajustar AutoARIMA para encontrar el mejor modelo ARIMA
        model = auto_arima(y, seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)
        future_predictions = model.predict(n_periods=12)
        
        # Crear las fechas de predicción a partir de enero de 2024
        future_dates = pd.date_range(start='2024-01-01', periods=12, freq='MS').strftime('%Y-%m-%d')
        
        for date, pred in zip(future_dates, future_predictions):
            predictions.append({
                'ID': id_value,
                'Fecha': date,
                'Prediccion_Venta': pred
            })
    except Exception as e:
        print(f'Error al ajustar el modelo para ID: {id_value} - {e}')
        continue

predictions_df = pd.DataFrame(predictions)
output_dir = r'C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo'
output_file = os.path.join(output_dir, 'AutoArima_Smooth.csv')
predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')

print(f'Predicciones guardadas en {output_file}')


C:\Users\etorres.DERCOPARTS\AppData\Local\Temp\ipykernel_2896\1693569103.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
C:\Users\etorres.DERCOPARTS\AppData\Local\Temp\ipykernel_2896\1693569103.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')  # Mantén el formato original


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=50.66 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=620.160, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=621.008, Time=0.08 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=619.756, Time=7.23 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=618.414, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=25.05 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 83.081 seconds
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=43.30 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=299.719, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=286.780, Time=0.17 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=284.241, Time=3.94 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=297.961, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=285.511, Time=8.54 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=285.856, Time=9.48 sec
